In [19]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a directory
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
    
# Hyperparameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

# MNIST dataset
dataset = torchvision.datasets.MNIST(root='../../data', train=True, transform = transforms.ToTensor(), download = True)

# Data Loader 
dataloader = torch.utils.data.DataLoader(dataset = dataset, batch_size = batch_size, shuffle=True)

# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var
    
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Start Training
for epoch in range(num_epochs):
    for i,(x, _) in enumerate(dataloader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute the reconstruction loss and kl divergence
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = -0.5*torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Reconst_Loss:{:4f}, KL_Loss:{:4f}'.
                  format(epoch+1, num_epochs, i+1,len(dataloader),reconst_loss.item(), kl_div.item()))
        
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))
        
        # Save the reconstructed image
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim = 3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.jpg'.format(epoch+1)))

Epoch [1/15], Step [50/469], Reconst_Loss:25947.347656, KL_Loss:691.180542
Epoch [1/15], Step [100/469], Reconst_Loss:22439.857422, KL_Loss:1356.777466
Epoch [1/15], Step [150/469], Reconst_Loss:19759.914062, KL_Loss:1666.079468
Epoch [1/15], Step [200/469], Reconst_Loss:17545.867188, KL_Loss:1925.825684
Epoch [1/15], Step [250/469], Reconst_Loss:16550.214844, KL_Loss:2077.678223
Epoch [1/15], Step [300/469], Reconst_Loss:15376.714844, KL_Loss:2229.704834
Epoch [1/15], Step [350/469], Reconst_Loss:14987.294922, KL_Loss:2473.872559
Epoch [1/15], Step [400/469], Reconst_Loss:13991.927734, KL_Loss:2422.817383
Epoch [1/15], Step [450/469], Reconst_Loss:14105.095703, KL_Loss:2578.634521
Epoch [2/15], Step [50/469], Reconst_Loss:13983.945312, KL_Loss:2769.368896
Epoch [2/15], Step [100/469], Reconst_Loss:12767.343750, KL_Loss:2848.732910
Epoch [2/15], Step [150/469], Reconst_Loss:13396.431641, KL_Loss:2685.715332
Epoch [2/15], Step [200/469], Reconst_Loss:12947.308594, KL_Loss:2851.184570
Ep

Epoch [12/15], Step [450/469], Reconst_Loss:10552.708008, KL_Loss:3283.437256
Epoch [13/15], Step [50/469], Reconst_Loss:10284.891602, KL_Loss:3266.072021
Epoch [13/15], Step [100/469], Reconst_Loss:10099.204102, KL_Loss:3211.196289
Epoch [13/15], Step [150/469], Reconst_Loss:9912.716797, KL_Loss:3221.292969
Epoch [13/15], Step [200/469], Reconst_Loss:10227.446289, KL_Loss:3265.600098
Epoch [13/15], Step [250/469], Reconst_Loss:10473.505859, KL_Loss:3086.606201
Epoch [13/15], Step [300/469], Reconst_Loss:9736.595703, KL_Loss:3141.135986
Epoch [13/15], Step [350/469], Reconst_Loss:10320.042969, KL_Loss:3175.619629
Epoch [13/15], Step [400/469], Reconst_Loss:10043.908203, KL_Loss:3169.038330
Epoch [13/15], Step [450/469], Reconst_Loss:10309.806641, KL_Loss:3183.256348
Epoch [14/15], Step [50/469], Reconst_Loss:9946.529297, KL_Loss:3295.073242
Epoch [14/15], Step [100/469], Reconst_Loss:10300.095703, KL_Loss:3260.750000
Epoch [14/15], Step [150/469], Reconst_Loss:10187.924805, KL_Loss:327